# Data Preprocessing

- [Dataset](https://www.kaggle.com/datasets/swaptr/turkey-earthquake-tweets)

In [3]:
import pandas as pd

# NOTE: unzip ./data/turkey_syria_earthquake_tweets/archive.zip before running the below code. `tweets.csv` file is too big.
full_data = pd.read_csv("./data/turkey_syria_earthquake_tweets/tweets.csv")
full_data.head()

In [1]:
# optional block for mounting to google drive directory (needed when running things in colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# optional block for mounting to google drive directory
import os
import pandas as pd
import numpy as np
import sys

GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'dl-twitter-crisis'
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

print(GOOGLE_DRIVE_PATH)
sys.path.append(GOOGLE_DRIVE_PATH)
df = pd.read_csv(GOOGLE_DRIVE_PATH + "/data/turkey_syria_earthquake_tweets/tweets_en.csv")

['.gitignore', 'README.md', 'environment.yml', '.DS_Store', 'paper', '.git', 'data', 'anomaly_detection.ipynb']
Running in google colab. Our path is `/content/drive/MyDrive/dl-twitter-crisis`
/content/drive/MyDrive/dl-twitter-crisis


In [13]:
# 1) Filtering English language: by using pandas, a Python data analysis library, in the language column, only rows where the language field had the value “en” were filtered.
# This step was necessary to increase the reliability of the pre-trained BERT model for sentiment analysis [ 36 ].
# After this filtering, 189,626 tweets out of 472,399 tweets were filtered as English text.

# NOTE: commenting this out as we already have the csv file

# print(len(full_data))
# en_filtered_data = full_data[full_data["language"] == "en"]
# print(len(en_filtered_data))
# en_filtered_data.to_csv(GOOGLE_DRIVE_PATH + "./data/turkey_syria_earthquake_tweets/tweets_en.csv")

In [11]:
# skip if using google colab
df = pd.read_csv("./data/turkey_syria_earthquake_tweets/tweets_en.csv")
df.head()

,Unnamed: 0,date,content,hashtags,like_count,rt_count,followers_count,isVerified,language,coordinates,place,source
0,1,2023-02-21 03:29:07+00:00,New search &amp; rescue work is in progress in...,"['Hatay', 'earthquakes', 'Türkiye', 'TurkiyeQu...",1.0,0.0,5697.0,True,en,NaN,NaN,Twitter Web App
1,2,2023-02-21 03:29:04+00:00,Can't imagine those who still haven't recovere...,"['Turkey', 'earthquake', 'turkeyearthquake2023...",0.0,0.0,1.0,False,en,NaN,NaN,Twitter for Android
2,3,2023-02-21 03:28:06+00:00,its a highkey sign for all of us to ponder ove...,"['turkeyearthquake2023', 'earthquake', 'Syria']",0.0,0.0,3.0,False,en,NaN,NaN,Twitter for Android
3,5,2023-02-21 03:27:27+00:00,"See how strong was the #Earthquake of Feb 20, ...","['Earthquake', 'Hatay', 'Turkey', 'turkeyearth...",0.0,0.0,21836.0,True,en,NaN,NaN,Twitter for Android
4,6,2023-02-21 03:27:11+00:00,More difficult news today on top of struggles ...,"['Türkiye', 'Syria', 'earthquake', 'Canadians']",1.0,0.0,675.0,False,en,NaN,NaN,Twitter for iPhone


In [3]:
# 2. Text lowercasing: all tweets were converted to lowercase; according to Hickman
# et al. [37 ], lowercasing tends to be beneficial because it reduces data dimensionality,
# thereby increasing statistical power, and usually does not reduce validity.
df['content'] = df['content'].str.lower()

In [4]:
# 3. Stop word removal: common English (function) words such as “and”, “is”, “I”, “am”,
# “what”, “of”, etc. were removed by using the Natural Language Toolkit (NLTK).
# Stop word removal has the advantages of reducing the size of the stored dataset and
# improving the overall efficiency and effectiveness of the analysis [38].

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt_tab')

# Get English stop words
stop_words = set(stopwords.words('english'))

# Function to remove stop words from text
def remove_stopwords(text):
    if pd.isna(text):
        return text

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words and return as string
    filtered_words = [word for word in words if word.lower() not in stop_words]

    return ' '.join(filtered_words)

df['content'] = df['content'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# 4. URLs removal: all URLs were removed from tweets, since the text of URL strings does
# not necessarily convey any relevant information, and can therefore be removed [39].

import re
df['content'] = df['content'].str.replace(r'http\S+', '', regex=True)

In [6]:
# 5. Duplicate removal: all duplicate tweets were removed to eliminate redundancy and
# possible skewing of the results.

df = df.drop_duplicates(subset='content', keep='first')

In [7]:
# exclude location info (96% of the tweets lacked geolocation)
# drop if exists
df = df.drop(columns=['coordinates', 'place', 'Unnamed: 0'], errors='ignore')
df.head()

,date,content,hashtags,like_count,rt_count,followers_count,isVerified,language,source
0,2023-02-21 03:29:07+00:00,new search & amp ; rescue work progress # hata...,"['Hatay', 'earthquakes', 'Türkiye', 'TurkiyeQu...",1.0,0.0,5697.0,True,en,Twitter Web App
1,2023-02-21 03:29:04+00:00,ca n't imagine still n't recovered previous tr...,"['Turkey', 'earthquake', 'turkeyearthquake2023...",0.0,0.0,1.0,False,en,Twitter for Android
2,2023-02-21 03:28:06+00:00,highkey sign us ponder actions return merciful...,"['turkeyearthquake2023', 'earthquake', 'Syria']",0.0,0.0,3.0,False,en,Twitter for Android
3,2023-02-21 03:27:27+00:00,"see strong # earthquake feb 20 , 2023 # hatay ...","['Earthquake', 'Hatay', 'Turkey', 'turkeyearth...",0.0,0.0,21836.0,True,en,Twitter for Android
4,2023-02-21 03:27:11+00:00,difficult news today top struggles already fac...,"['Türkiye', 'Syria', 'earthquake', 'Canadians']",1.0,0.0,675.0,False,en,Twitter for iPhone


In [8]:
# Tokenize inputs
!pip install -U transformers

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# pipe = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

inputs = df['content'].tolist()

# train, test, val split
# TODO: figure out a way to downsample
# train_size = int(0.8 * len(inputs))
# val_size = int(0.1 * len(inputs))
# test_size = len(inputs) - train_size - val_size

# train_inputs, val_inputs, test_inputs = inputs[:train_size], inputs[train_size:train_size+val_size], inputs[train_size+val_size:]

example_inputs = inputs[:100]
model_inputs = tokenizer(example_inputs, padding=True, truncation=True, max_length=128, return_tensors="pt")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [9]:
# Call BERT and get predicted labels
outputs = model(**model_inputs)
logits = outputs.logits
probabilities = torch.softmax(logits, dim=-1)
predicted_labels = torch.argmax(probabilities, dim=-1)
print(predicted_labels)

tensor([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 4, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
        0, 0, 0, 0])


# Neural Network Models

- Sentiment Analysis
  - pre-trained transformer-based `BERT` model

- Anomaly Detection
  - `autoencoder`
  - `LSTM with Attention`

## Sentiment Analysis
- `nlptown/bert-base-multilingual-uncased-sentiment` :  fine-tuned version of `bert-base-multilingual-uncased`, which is optimized for sentiment analysis across six languages: English, Dutch, German, French, Spanish and Italian.
- Reference: Lakhanpal, S.; Gupta, A.; Agrawal, R. Leveraging Explainable AI to Analyze Researchers’ Aspect-Based Sentiment About ChatGPT. In Proceedings of the 15th International Conference on Intelligent Human Computer Interaction (IHCI 2023), Daegu, Republic of Korea, 8–10 November 2023; pp. 281–290.

- Can be seen as part of preprocessing???


In [ ]:
# TODO: Tweets were tokenized using the AutoTokenizer from HuggingFace Transformers, truncated to a maximum length of 512 tokens [41].

# TODO: The model predicted sentiment scores across five classes representing very negative to very positive sentiments.
# These categorical outputs were then converted to a continuous polarity scale ranging from −1 (strongly negative) to +1 (strongly positive) to facilitate the temporal analysis of sentiment fluctuations

## Anomaly Detection

- `autoencoder`
  - An autoencoder neural network was designed and trained to detect anomalies based on deviations in tweet sentiment patterns.
  - The input data was structured into sequences of polarity scores.
  - The autoencoder was implemented as a fully connected feedforward network with a three-layer encoder and symmetric decoder.
  - The encoder consisted of a hidden layer with 64 neurons followed by a 16-neuron bottleneck, using rectified linear unit (ReLU) activations for encoding and decoding [ 42 ].
  - Reconstruction errors (mean squared error between actual and reconstructed sequences) were calculated, and tweets with errors above the 95th percentile threshold were flagged as anomalies.

- `LSTM with Attention`
  - An LSTM neural network with an integrated attention mechanism was implemented to detect anomalies based on prediction errors.
  - Input sequences of polarity scores were processed through LSTM layers, and attention layers were applied to selectively weigh temporal dependencies within the sequences.
  - The LSTM with attention included a single-layer LSTM model with a hidden size of 32, followed by an attention mechanism.

- Common config
  - Both models were trained for 10 epochs using the Adam optimizer (learning rate was set to 0.001), with a batch size of 32 and mean squared error (MSE) loss.
  - Sentiment polarity scores were normalized using MinMax scaling to the [0,1] range. The model’s output was a prediction of subsequent sentiment scores.
  - Anomalies were identified when prediction errors exceeded a threshold set at the 95th percentile, highlighting sudden or extreme shifts (changes) in sentiment.